In [4]:
import json
def add_source_to_existing_json(qa_list):
    # 출처 매핑 테이블
    source_map = [
        (1, 20, "https://terms.naver.com/entry.naver?docId=3569533&cid=59015&categoryId=59015"),
        (21, 40, "https://terms.naver.com/entry.naver?docId=5707247&cid=43667&categoryId=43667"),
        (41, 70, "https://encykorea.aks.ac.kr/Article/E0029857"),
        (71, 90, "https://encykorea.aks.ac.kr/Article/E0065805"),
        (91, 105, "https://encykorea.aks.ac.kr/Article/E0048697"),
        (106, 125, "https://encykorea.aks.ac.kr/Article/E0065420"),
        (126, 145, "https://encykorea.aks.ac.kr/Article/E0029365"),
        (146, 165, "https://encykorea.aks.ac.kr/Article/E0055081")
    ]

    result = []
    for idx, qa in enumerate(qa_list, start=1):
        # 출처 URL 매핑
        source = next((url for start, end, url in source_map if start <= idx <= end), "출처 없음")

        # 기존 QA에 번호와 출처 추가
        enriched = {
            "number": idx,
            "question": qa["question"],
            "answer": qa["answer"],
            "source": source
        }
        result.append(enriched)

    return result


# 파일에서 읽기
with open('data.json', 'r', encoding='utf-8') as f:
    qa_list = json.load(f)

# 위 함수 적용
updated = add_source_to_existing_json(qa_list)

# 저장
with open('qa_with_source.json', 'w', encoding='utf-8') as f:
    json.dump(updated, f, ensure_ascii=False, indent=2)

# ✅ 실행

updated_qa_list = add_source_to_existing_json(qa_list)

# ✅ 결과 출력
print(json.dumps(updated_qa_list, indent=2, ensure_ascii=False))

[
  {
    "number": 1,
    "question": "세종대왕은 어떤 인물인가요?",
    "answer": "세종대왕은 조선의 제4대 왕으로, 한글을 창제하고 유교 정치와 민족문화를 꽃피운 인물입니다. 당대에는 '해동요순'이라 불릴 만큼 존경받았으며, 지금까지도 가장 위대한 왕 중 하나로 평가받습니다.",
    "source": "https://terms.naver.com/entry.naver?docId=3569533&cid=59015&categoryId=59015"
  },
  {
    "number": 2,
    "question": "세종대왕은 어떤 호칭으로 불렸나요?",
    "answer": "세종대왕은 '성군', '대왕'이라는 호칭으로 불렸고, 당대에는 '해동요순'이라는 이상적 성군의 대명사로도 칭해졌습니다.",
    "source": "https://terms.naver.com/entry.naver?docId=3569533&cid=59015&categoryId=59015"
  },
  {
    "number": 3,
    "question": "세종대왕의 본명과 자는 무엇인가요?",
    "answer": "세종대왕의 이름은 이도(李祹)이며, 자는 원정(元正)입니다.",
    "source": "https://terms.naver.com/entry.naver?docId=3569533&cid=59015&categoryId=59015"
  },
  {
    "number": 4,
    "question": "세종대왕의 즉위 과정은 어땠나요?",
    "answer": "세종은 셋째 아들이었지만, 태종이 세자 이제를 폐위하고 충녕대군이었던 세종을 세자로 책봉하며 왕위에 올랐습니다. 태종은 총명하고 학문을 좋아한 세종의 자질을 높이 평가했습니다.",
    "source": "https://terms.naver.com/entry.naver?docId=3569533&cid=59015&categoryId=59015"

In [5]:
# 파일에서 읽기
with open('data.json', 'r', encoding='utf-8') as f:
    qa_list = json.load(f)

print(f"✅ 원본 QA 개수: {len(qa_list)}")

# 위 함수 적용
updated = add_source_to_existing_json(qa_list)

print(f"✅ 출처 추가 후 QA 개수: {len(updated)}")

# 저장
with open('qa_with_source.json', 'w', encoding='utf-8') as f:
    json.dump(updated, f, ensure_ascii=False, indent=2)

✅ 원본 QA 개수: 164
✅ 출처 추가 후 QA 개수: 164


In [16]:
import json
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import itertools

# 1. JSON 데이터 로드
with open('qa_with_source.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

questions = [item["question"] for item in data]

# 2. TF-IDF 벡터화
vectorizer = TfidfVectorizer().fit_transform(questions)
cosine_sim = cosine_similarity(vectorizer)

# 3. 유사도가 높은 쌍 출력 (예: 0.85 이상)
threshold = 0.60
print("유사한 질문 쌍 (유사도 {:.2f} 이상):".format(threshold))
printed = set()

for i, j in itertools.combinations(range(len(questions)), 2):
    sim = cosine_sim[i, j]
    if sim >= threshold:
        pair = tuple(sorted((i, j)))
        if pair not in printed:
            printed.add(pair)
            print(f"[{i+1}] {questions[i]}\n[{j+1}] {questions[j]}\n→ 유사도: {sim:.4f}\n")

유사한 질문 쌍 (유사도 0.60 이상):
[34] 4군 6진 개척의 목적은 무엇이었나요?
[59] 4군 6진 개척의 의의는 무엇인가요?
→ 유사도: 0.6619

[36] 장영실은 어떤 업적을 남겼나요?
[38] 맹사성은 어떤 업적을 남겼나요?
→ 유사도: 0.6666

[49] 세종 시대의 대표적인 물시계는 무엇인가요?
[51] 세종 시대의 대표적인 금속활자는 무엇인가요?
→ 유사도: 0.6926

[115] 황희는 어느 지역에서 제향되고 있나요?
[134] 성삼문은 어느 지역에서 제향되고 있나요?
→ 유사도: 0.8693

[124] 성삼문이 훈민정음 창제에 어떤 역할을 했나요?
[127] 성삼문이 관직에서 어떤 역할을 했나요?
→ 유사도: 0.6388



In [15]:
import json

# 기존 JSON 데이터를 불러온다고 가정 (예: data.json에서 로딩)
with open("qa_with_source.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# number 필드 재정렬
for i, item in enumerate(data, start=1):
    item["number"] = i

# 결과 저장 (선택)
with open("qa_with_source.json", "w", encoding="utf-8") as f:
    json.dump(data, f, ensure_ascii=False, indent=2)

# 결과 출력 확인
# print(json.dumps(data[:5], ensure_ascii=False, indent=2))  # 처음 5개만 미리보기

In [19]:
import os
os.environ["TRANSFORMERS_NO_TF"] = "1"  # TensorFlow 비활성화

from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer("snunlp/KR-SBERT-V40K-klueNLI-augSTS")
from sentence_transformers import SentenceTransformer, util
import itertools

model = SentenceTransformer("snunlp/KR-SBERT-V40K-klueNLI-augSTS")

# 질문만 추출
questions = [item["question"] for item in data]
embeddings = model.encode(questions, convert_to_tensor=True)

# 유사도 계산
pairs = list(itertools.combinations(range(len(questions)), 2))
results = []

for i, j in pairs:
    score = util.cos_sim(embeddings[i], embeddings[j]).item()
    if score > 0.85:
        results.append((i, j, score))

# 유사 질문 출력
for i, j, score in sorted(results, key=lambda x: -x[2]):
    print(f"\n[유사도: {score:.3f}]")
    print(f"Q{i+1}: {questions[i]}")
    print(f"Q{j+1}: {questions[j]}")


[유사도: 0.939]
Q91: 장영실이 만든 최초의 물시계는 무엇인가요?
Q92: 장영실이 제작한 대표적인 물시계는 무엇인가요?

[유사도: 0.915]
Q67: 훈민정음 해례본은 무엇인가요?
Q71: 훈민정음 해례본은 어디에서 발견되었나요?

[유사도: 0.903]
Q147: 집현전의 제1기(1420~1427)의 특징은?
Q148: 집현전의 제2기(1428~1436)는 어떤 시기였나요?

[유사도: 0.898]
Q70: 훈민정음 해례본은 언제 간행되었나요?
Q71: 훈민정음 해례본은 어디에서 발견되었나요?

[유사도: 0.894]
Q34: 4군 6진 개척의 목적은 무엇이었나요?
Q59: 4군 6진 개척의 의의는 무엇인가요?

[유사도: 0.894]
Q26: 훈민정음은 언제 창제되고 반포되었나요?
Q45: 훈민정음은 어떤 배경에서 창제되었나요?

[유사도: 0.889]
Q67: 훈민정음 해례본은 무엇인가요?
Q70: 훈민정음 해례본은 언제 간행되었나요?

[유사도: 0.883]
Q67: 훈민정음 해례본은 무엇인가요?
Q85: 훈민정음 해례본의 학술적 가치는 무엇인가요?

[유사도: 0.882]
Q67: 훈민정음 해례본은 무엇인가요?
Q68: 훈민정음 해례본의 구성은 어떻게 되나요?

[유사도: 0.875]
Q1: 세종대왕은 어떤 인물인가요?
Q21: 세종대왕은 몇 대 왕인가요?

[유사도: 0.874]
Q14: 훈민정음 창제 목적은 무엇인가요?
Q45: 훈민정음은 어떤 배경에서 창제되었나요?

[유사도: 0.862]
Q67: 훈민정음 해례본은 무엇인가요?
Q69: 훈민정음 해례본의 집필자는 누구인가요?

[유사도: 0.862]
Q48: 앙부일구는 어떤 해시계인가요?
Q95: 앙부일구는 어떤 시계인가요?

[유사도: 0.855]
Q23: 세종대왕의 왕비는 누구였나요?
Q39: 세종대왕은 어떻게 즉위하게 되었나요?

[유사도: 0.854]
Q106: 황희는 국가 제도 개혁에 어떤 기여를 했나요?
Q108: 황희는 어떤 법제 개혁을 했나요?

[유사도: 0.85

In [20]:
from collections import defaultdict

clusters = defaultdict(list)
keywords = {
    "훈민정음": ["훈민정음", "해례본"],
    "장영실": ["장영실", "혼천의", "자격루", "천문", "시계"],
    "황희": ["황희"],
    "성삼문": ["성삼문", "사육신"],
    "집현전": ["집현전"],
    "과학기술": ["과학", "기술", "측우기", "도량형", "금속활자"],
    "정치제도": ["정치", "의정부서사제", "육조직계제", "공법"],
    "외교군사": ["여진", "왜구", "4군6진", "대마도", "쓰시마", "정벌"],
}

for item in data:
    assigned = False
    for topic, kws in keywords.items():
        if any(kw in item["question"] for kw in kws):
            clusters[topic].append(item)
            assigned = True
            break
    if not assigned:
        clusters["기타"].append(item)

# 결과 확인
for topic, items in clusters.items():
    print(f"\n[주제: {topic}] - {len(items)}개 문항")
    for i in items:
        print(f"  - {i['question']}")


[주제: 기타] - 48개 문항
  - 세종대왕은 어떤 인물인가요?
  - 세종대왕은 어떤 호칭으로 불렸나요?
  - 세종대왕의 본명과 자는 무엇인가요?
  - 세종대왕의 즉위 과정은 어땠나요?
  - 즉위 초기에 세종은 어떤 어려움을 겪었나요?
  - 세종대왕은 어떤 학문적 특징을 가졌나요?
  - 세종은 어떤 방식으로 제도를 만들었나요?
  - 세종대왕은 어떤 역사서를 정비했나요?
  - 세종 시기에 인쇄술은 어떻게 발전했나요?
  - 세종은 음악 분야에서도 어떤 업적을 남겼나요?
  - 세종대왕의 애민정신은 어떻게 나타났나요?
  - 세종대왕은 어떤 병을 앓았나요?
  - 세종대왕은 언제 세상을 떠났나요?
  - 세종대왕은 언제부터 언제까지 재위했나요?
  - 세종대왕은 몇 대 왕인가요?
  - 세종대왕의 출생과 사망 연도는?
  - 세종대왕의 왕비는 누구였나요?
  - 세종대왕의 능은 어디에 있나요?
  - 세종대왕이 장려한 한글 문헌에는 어떤 것들이 있나요?
  - 세종은 음악 분야에서 어떤 조치를 했나요?
  - 칠정산은 무엇인가요?
  - 세종대왕이 시행한 조세 제도는?
  - 세종은 민생 안정에 어떤 노력을 했나요?
  - 세종대왕의 대외 정책은 무엇이 있었나요?
  - 4군 6진 개척의 목적은 무엇이었나요?
  - 맹사성은 어떤 업적을 남겼나요?
  - 세종대왕은 어떻게 즉위하게 되었나요?
  - 세종은 유교적 의례 정비에서 어떤 태도를 취했나요?
  - 세종대의 편찬사업에는 어떤 책들이 있었나요?
  - 총통등록은 어떤 책인가요?
  - 농사직설은 어떤 책인가요?
  - 세종이 편찬한 의약서는 어떤 것이 있나요?
  - 정간보는 어떤 기보법인가요?
  - 세종이 작곡한 곡에는 무엇이 있나요?
  - 세종 시대의 법제 정비 중 대표적인 것은?
  - 4군 6진 개척의 의의는 무엇인가요?
  - 세종의 불교 정책은 어땠나요?
  - 세종은 어떤 인성의 왕이었나요?
  - 세종은 사료 관리에 어떤 제도를 도입했나요?
  - 세종의 묘호와 시호는 

In [22]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# 테스트 후보 모델 목록
model_names = [
    "jhgan/ko-sbert-nli",
    "BM-K/KoSimCSE-roberta",
    "intfloat/multilingual-e5-base"
]

# 테스트용 쿼리와 정답
query = "세종대왕은 어떤 업적을 남겼나요?"
corpus = [
    "Q: 세종대왕은 어떤 인물인가요? A: 조선의 제4대 왕으로 한글을 창제하고...",
    "Q: 훈민정음의 제자 원리는 무엇인가요? A: 상형과 음양오행에 기반하여...",
    "Q: 세종 시대의 과학기술 발명은 무엇인가요? A: 자격루, 혼천의, 측우기 등..."
]
reference_answer = corpus[0]  # 정답 텍스트

# 모델별 유사도 평가
for model_name in model_names:
    print(f"\n🔍 Testing model: {model_name}")
    model = SentenceTransformer(model_name)
    
    query_vec = model.encode([query])
    corpus_vecs = model.encode(corpus)
    
    scores = cosine_similarity(query_vec, corpus_vecs)[0]
    
    # 상위 문장 및 유사도
    for idx, score in enumerate(scores):
        print(f"  → 문장 {idx+1} 유사도: {score:.4f} | {corpus[idx]}")
    
    best_idx = np.argmax(scores)
    print(f"✅ Best match: #{best_idx+1} | Score: {scores[best_idx]:.4f}")


🔍 Testing model: jhgan/ko-sbert-nli
  → 문장 1 유사도: 0.7800 | Q: 세종대왕은 어떤 인물인가요? A: 조선의 제4대 왕으로 한글을 창제하고...
  → 문장 2 유사도: 0.5680 | Q: 훈민정음의 제자 원리는 무엇인가요? A: 상형과 음양오행에 기반하여...
  → 문장 3 유사도: 0.6642 | Q: 세종 시대의 과학기술 발명은 무엇인가요? A: 자격루, 혼천의, 측우기 등...
✅ Best match: #1 | Score: 0.7800

🔍 Testing model: BM-K/KoSimCSE-roberta


No sentence-transformers model found with name BM-K/KoSimCSE-roberta. Creating a new one with mean pooling.


  → 문장 1 유사도: 0.7422 | Q: 세종대왕은 어떤 인물인가요? A: 조선의 제4대 왕으로 한글을 창제하고...
  → 문장 2 유사도: 0.3908 | Q: 훈민정음의 제자 원리는 무엇인가요? A: 상형과 음양오행에 기반하여...
  → 문장 3 유사도: 0.6262 | Q: 세종 시대의 과학기술 발명은 무엇인가요? A: 자격루, 혼천의, 측우기 등...
✅ Best match: #1 | Score: 0.7422

🔍 Testing model: intfloat/multilingual-e5-base
  → 문장 1 유사도: 0.8622 | Q: 세종대왕은 어떤 인물인가요? A: 조선의 제4대 왕으로 한글을 창제하고...
  → 문장 2 유사도: 0.7571 | Q: 훈민정음의 제자 원리는 무엇인가요? A: 상형과 음양오행에 기반하여...
  → 문장 3 유사도: 0.8232 | Q: 세종 시대의 과학기술 발명은 무엇인가요? A: 자격루, 혼천의, 측우기 등...
✅ Best match: #1 | Score: 0.8622


In [5]:
from sentence_transformers import SentenceTransformer, util
import torch
"""
    "jhgan/ko-sbert-nli",
    "BM-K/KoSimCSE-roberta",
    "intfloat/multilingual-e5-base",
    "upskyy/bge-m3-korean",
    "jhgan/ko-sroberta-multitask",
    "nlpai-lab/KoE5",
    """
# 테스트할 모델 목록
model_ids = [
    "intfloat/multilingual-e5-base",
    "Qwen/Qwen3-Embedding-0.6B",
    "intfloat/multilingual-e5-large-instruct",
]

# 검증용 질문-답변 세트
qa_pairs = [
    {
        "question": "세종은 정치 권력을 어떻게 분산시키며 유교적 이상을 실현했나요?",
        "answer": "건강상의 이유로 의정부 서사제로 전환하고 세자에게 정무를 위임하여, 권력을 분산시키는 유교적 왕도 정치를 실현했습니다."
    },
    {
        "question": "훈민정음의 제자 원리가 음양오행과 어떤 철학적 관계가 있나요?",
        "answer": "제자해에서는 자음·모음 제자 원리를 설명하며, 조음위치 상형, 음양오행 이론, 36자모 체계 등 음운 철학이 반영되어 있습니다."
    },
    {
        "question": "세종 시대의 천문 기구들은 어떤 원리를 기반으로 제작되었으며, 백성 생활에 어떤 영향을 미쳤나요?",
        "answer": "세종은 혼천의, 정방안, 간의대 등을 설치하고 천문 관측을 체계화했으며, 백성들이 시간을 알 수 있도록 앙부일구와 자격루를 제작하여 일상생활에 도움을 주었습니다."
    }
]

# 유사도 평가
for model_id in model_ids:
    model = SentenceTransformer(model_id)
    print(f"\nTesting model: {model_id}")
    
    for pair in qa_pairs:
        q_vec = model.encode(pair["question"], convert_to_tensor=True)
        a_vec = model.encode(pair["answer"], convert_to_tensor=True)
        similarity = util.cos_sim(q_vec, a_vec).item()
        print(f"  → Q: {pair['question']}\n    A: {pair['answer']}\n    유사도: {similarity:.4f}")


Testing model: intfloat/multilingual-e5-base
  → Q: 세종은 정치 권력을 어떻게 분산시키며 유교적 이상을 실현했나요?
    A: 건강상의 이유로 의정부 서사제로 전환하고 세자에게 정무를 위임하여, 권력을 분산시키는 유교적 왕도 정치를 실현했습니다.
    유사도: 0.8596
  → Q: 훈민정음의 제자 원리가 음양오행과 어떤 철학적 관계가 있나요?
    A: 제자해에서는 자음·모음 제자 원리를 설명하며, 조음위치 상형, 음양오행 이론, 36자모 체계 등 음운 철학이 반영되어 있습니다.
    유사도: 0.8535
  → Q: 세종 시대의 천문 기구들은 어떤 원리를 기반으로 제작되었으며, 백성 생활에 어떤 영향을 미쳤나요?
    A: 세종은 혼천의, 정방안, 간의대 등을 설치하고 천문 관측을 체계화했으며, 백성들이 시간을 알 수 있도록 앙부일구와 자격루를 제작하여 일상생활에 도움을 주었습니다.
    유사도: 0.8743


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/215 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/16.3k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/271 [00:00<?, ?B/s]


Testing model: intfloat/multilingual-e5-large-instruct
  → Q: 세종은 정치 권력을 어떻게 분산시키며 유교적 이상을 실현했나요?
    A: 건강상의 이유로 의정부 서사제로 전환하고 세자에게 정무를 위임하여, 권력을 분산시키는 유교적 왕도 정치를 실현했습니다.
    유사도: 0.8951
  → Q: 훈민정음의 제자 원리가 음양오행과 어떤 철학적 관계가 있나요?
    A: 제자해에서는 자음·모음 제자 원리를 설명하며, 조음위치 상형, 음양오행 이론, 36자모 체계 등 음운 철학이 반영되어 있습니다.
    유사도: 0.8978
  → Q: 세종 시대의 천문 기구들은 어떤 원리를 기반으로 제작되었으며, 백성 생활에 어떤 영향을 미쳤나요?
    A: 세종은 혼천의, 정방안, 간의대 등을 설치하고 천문 관측을 체계화했으며, 백성들이 시간을 알 수 있도록 앙부일구와 자격루를 제작하여 일상생활에 도움을 주었습니다.
    유사도: 0.8803


In [26]:
from sentence_transformers import SentenceTransformer
import faiss
import json
import numpy as np

# 1. JSON 로딩
with open("qa_with_source.json", "r", encoding="utf-8") as f:
    qa_data = json.load(f)

# 2. 질문+답변 텍스트 생성
corpus = [f"Q: {item['question']} A: {item['answer']}" for item in qa_data]

# 3. 임베딩 모델 로딩
model = SentenceTransformer("intfloat/multilingual-e5-base")

# 4. 임베딩 계산
embeddings = model.encode(corpus, show_progress_bar=True, convert_to_numpy=True)
embeddings = embeddings / np.linalg.norm(embeddings, axis=1, keepdims=True)  # 코사인 정규화

# 5. FAISS 인덱스 생성
dimension = embeddings.shape[1]
index = faiss.IndexFlatIP(dimension)
index.add(embeddings)

# 6. 인덱스 및 메타데이터 저장
faiss.write_index(index, "sejong_qa_faiss.index")

metadata = {i: {
    "number": qa_data[i]["number"],
    "question": qa_data[i]["question"],
    "answer": qa_data[i]["answer"]
} for i in range(len(qa_data))}

with open("sejong_qa_metadata.json", "w", encoding="utf-8") as f:
    json.dump(metadata, f, ensure_ascii=False, indent=2)

Batches: 100%|██████████| 5/5 [00:04<00:00,  1.07it/s]


In [ ]:
from sentence_transformers import SentenceTransformer
import faiss
import json
import numpy as np

# 로딩
index = faiss.read_index("sejong_qa_faiss.index")
with open("sejong_qa_metadata.json", "r", encoding="utf-8") as f:
    metadata = json.load(f)

model = SentenceTransformer("intfloat/multilingual-e5-base")

# 검색 함수
def search(query, top_k=3):
    query_embedding = model.encode([query])
    query_embedding = query_embedding / np.linalg.norm(query_embedding, axis=1, keepdims=True)

    D, I = index.search(query_embedding, top_k)
    for rank, idx in enumerate(I[0]):
        result = metadata[str(idx)]
        print(f"\n🔎 TOP {rank+1} (score: {D[0][rank]:.4f})")
        print(f"Q: {result['question']}")
        print(f"A: {result['answer']}")

# 예시 질의
search("세종은 권력을 어떻게 나누었나요?")


🔎 TOP 1 (score: 0.8606)
Q: 세종은 어떤 정치체제를 실시했나요?
A: 세종은 의정부서사제를 시행하여 재상이 정치를 주도하고, 왕은 군사와 인사에 집중하는 방식으로 왕권과 신권의 조화를 이루었습니다.

🔎 TOP 2 (score: 0.8425)
Q: 세종대왕의 즉위 과정은 어땠나요?
A: 세종은 셋째 아들이었지만, 태종이 세자 이제를 폐위하고 충녕대군이었던 세종을 세자로 책봉하며 왕위에 올랐습니다. 태종은 총명하고 학문을 좋아한 세종의 자질을 높이 평가했습니다.

🔎 TOP 3 (score: 0.8376)
Q: 세종은 후반기에 어떤 정치적 조치를 했나요?
A: 건강상의 이유로 의정부 서사제로 전환하고 세자에게 정무를 위임하여, 권력을 분산시키는 유교적 왕도 정치를 실현했습니다.


질문: 한글의 창제 원리는 무엇인가요?
답변: 짐이 백성의 편리를 도모하고자 하여, 한글을 창제하였도다. 어리석고 지혜로운 이를 불문하고, 모든 백성이 쉽게 글을 읽고 쓸 수 있도록, 한글의 자모를 만들었느니라.

한글의 창제 원리는, 먼저 말소리와 글자체를 연구하였도다. 우리 백성의 말소리를 바탕으로, 새로운 글자체를 만들었느니라. 한글은十四個의 자모로 구성되었으며, 초성과 중성, 종성을 합하여 하나의 글자를 이루었도다.

짐이 백성의 편리를 위하여, 한글을 창제하였으니, 백성들이 쉽게 글을 읽고 쓰고, 학문에 뜻을 두어, 지혜를 넓히도록 하였느니라. 이를 위하여, 『훈민정음』을 편찬하였도다. 이 책은 한글의 정체와 소리를 가르치고, 백성들이 쉽게 이해할 수 있도록 하였느니라.

한글의 창제는, 짐이 백성의 복지를 위하여, 심혈을 기울인 사업이었도다. 백성들이 한글을 익히고, 이를 통해 학문과 지혜를 넓혀, 더好的 삶을 살 수 있도록 하였느니라. 짐이 한글을 창제한 것은, 백성의 애민을 위한 일이니, 후세에까지 전해질 것이라 믿도다.

질문: 김치의 역사와 문화적 중요성에 대해 설명해주세요.
답변: 짐이 김치의 역사와 문화적 중요성을 설명하여 백성們에게 김치의 가치를 널리 알리도록 하겠습니다.

김치는 우리나라의 전통 음식으로, 신선한 채소와 다양한 양념을 조합하여 만든 발효 음식이니라. 김치의 역사는 우리나라의 역사만큼悠久하여, 삼국 시대 이전부터 김치와 같은 발효 음식을 먹었다는 기록이 있으므로, 김치의 역사도 매우 길¯Â다고 하겠도다.

김치는 우리나라의 대표적인 식품으로, 각종 채소와 다양한 양념을 사용하여 만든 음식이니라. 김치의 종류는 백김치, 깍두기, 김치전, 오이소박이 등 매우 다양하여, 각 지역과 계절에 따라 다양한 김치를 만들 수 있느니라.

김치는 단순한 음식이 아니라, 우리나라의 문화와 전통을 상징하는 음식이니라. 김치는 우리의 조상들이 건강하고 맛있는 음식을 만들기 위해 노력한 결과로, 우리나라의 식문화를 대표하는 음식이라고 하겠도다.

RuntimeError: MPS backend out of memory (MPS allocated: 17.80 GB, other allocations: 2.98 MB, max allowed: 18.13 GB). Tried to allocate 732.43 MB on private pool. Use PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 to disable upper limit for memory allocations (may cause system failure).

짐은 학문과 독서를甚은 즐겨 하였도다. 특히 역사와 천문학, 음악 등에 큰 관심을 가졌으며, 백성들의 삶을 улучш하기 위한 과학기술에도 많은 관심을 두었느니라. 이에 대해 더 알고 싶은 것이 있는가?
